# 제 1유형

In [16]:
import pandas as pd

iris = pd.read_csv('iris.csv')
SW = iris['Sepal.Width']
SW_mean = SW.mean()
SW_std = SW.std()
under_bound = SW_mean - 3*SW_std
over_bound = SW_mean + 3*SW_std
result_list = []
# print(under_bound, over_bound)

print(SW[(SW <= under_bound) | (SW >= over_bound)].sum())

4.4


In [79]:
import pandas as pd

mtcars = pd.read_csv('mtcars1.csv')
disp = mtcars['disp']
# print(disp)

sorted_mtcars = mtcars.sort_values(by='disp', ascending=False)
rank_series = sorted_mtcars['disp'].rank(ascending=False, method='min')

sorted_mtcars['rank'] = rank_series

print(round(sorted_mtcars[sorted_mtcars['rank'] <= 20]['disp'].std(), 2))

97.47


In [89]:
import pandas as pd

cars = pd.read_csv('Cars93.csv')

# print(cars.info())

all_record = len(cars)
# print(all_record)

na_record = cars.isna().sum()
na_col_record = na_record[na_record >= 1].count()
# print(na_col_record)

all_na_record = na_record.sum()
# print(all_na_record)

remove_na_record = len(cars[na_record[na_record >= 10].index].dropna())
# print(remove_na_record)

two_na_record = sum(cars[cars.isna().sum(axis=1) >= 2].index)
# print(two_na_record)

print(all_record + na_col_record + all_na_record + remove_na_record + two_na_record)

1090


# 제 2유형

In [ ]:
import pandas as pd

X_train = pd.read_csv()

# 제 3유형

In [108]:
import pandas as pd
from scipy.stats import *

cars = pd.read_csv('Cars93.csv')
# print(cars)

small = cars[cars['Type'] == 'Small']['Price'].dropna()
large = cars[cars['Type'] == 'Large']['Price'].dropna()

mu1 = large.mean()
mu2 = small.mean()

# (a)
print(round(mu1 - mu2, 2))


stat, pvalue = ttest_ind(large, small, equal_var=True)

# stat = (mu1 - mu2) / se
se = (mu1 - mu2) / stat

# (b)
print(round(se, 2))

dof = len(small) + len(large) - 2
lower_cl = round(t.interval(0.95, dof, mu1-mu2, se)[0], 1)
print(lower_cl)
print('기각')

13.85
1.61
10.5
기각
